In [1]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score, confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('data/car_evaluation.csv', header=None, names=['buying','maintenance','doors','persons','lug_boot','safety','decision'])
df.head()

,buying,maintenance,doors,persons,lug_boot,safety,decision
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   buying       1728 non-null   object
 1   maintenance  1728 non-null   object
 2   doors        1728 non-null   object
 3   persons      1728 non-null   object
 4   lug_boot     1728 non-null   object
 5   safety       1728 non-null   object
 6   decision     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [28]:
df['buying'].value_counts()

med      432
low      432
vhigh    432
high     432
Name: buying, dtype: int64

In [29]:
df['decision'].value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: decision, dtype: int64

In [60]:
df['persons'].value_counts()

more    576
2       576
4       576
Name: persons, dtype: int64

In [30]:
get_d=pd.get_dummies(df[['buying','maintenance','lug_boot','safety']],drop_first=True)

In [31]:
df = pd.concat([df,get_d], axis=1)

In [32]:
df

,buying,maintenance,doors,persons,lug_boot,safety,decision,buying_low,buying_med,buying_vhigh,maintenance_low,maintenance_med,maintenance_vhigh,lug_boot_med,lug_boot_small,safety_low,safety_med
0,vhigh,vhigh,2,2,small,low,unacc,0,0,1,0,0,1,0,1,1,0
1,vhigh,vhigh,2,2,small,med,unacc,0,0,1,0,0,1,0,1,0,1
2,vhigh,vhigh,2,2,small,high,unacc,0,0,1,0,0,1,0,1,0,0
3,vhigh,vhigh,2,2,med,low,unacc,0,0,1,0,0,1,1,0,1,0
4,vhigh,vhigh,2,2,med,med,unacc,0,0,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1,0,0,1,0,0,1,0,0,1
1724,low,low,5more,more,med,high,vgood,1,0,0,1,0,0,1,0,0,0
1725,low,low,5more,more,big,low,unacc,1,0,0,1,0,0,0,0,1,0
1726,low,low,5more,more,big,med,good,1,0,0,1,0,0,0,0,0,1


In [33]:
df = df.drop(['buying','maintenance','lug_boot','safety'], axis=1)

In [35]:
from sklearn.preprocessing import LabelEncoder
y=df['decision']
label_encoder = LabelEncoder().fit(y)
y = label_encoder.transform(y)

In [42]:
np.unique(y,return_counts=True )

(array([0, 1, 2, 3]), array([ 384,   69, 1210,   65], dtype=int64))

In [51]:
X=df.drop('decision', axis=1)



In [46]:
class DecisionTree_Classifier:
    def __init__(self):
        self.DTC = DecisionTreeClassifier()
    
    def get_params_for_dtc(self, train_x, train_y):
        try:
            self.param_grid_for_dtc={"criterion":['gini','entropy'], 'max_depth':range(1,10),
                                     "min_samples_split": range(1,10), "min_samples_leaf":range(1,5),
                                     "max_features":['auto', 'sqrt','log2']}
            self.str_kfold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
            self.grid = GridSearchCV(estimator=self.DTC, param_grid=self.param_grid_for_dtc, verbose=0, cv=self.str_kfold_cv)
            self.grid.fit(train_x,train_y)
            
            self.criterions = self.grid.best_params_['criterion']
            self.max_depth = self.grid.best_params_['max_depth']
            self.min_samples_split = self.grid.best_params_['min_samples_split']
            self.min_samples_leaf = self.grid.best_params_['min_samples_leaf']
            self.max_features = self.grid.best_params_['max_features']
            
            self.dtc = DecisionTreeClassifier(criterion=self.criterions, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
                                              min_samples_leaf = self.min_samples_leaf, max_features=self.max_features)
            self.dtc.fit(train_x,train_y)
            print(f'DecisionTreeClassifier best params:{str(self.grid.best_params_)}.')
            return self.dtc
       
        except Exception as e:
            print(f'Exception occured in get_params_for_dtc method. Exception message: {str(e)}')
            print('LogisticReg Parameter tuning  failed.')
            
    def dtc_model(self, train_x, test_x, train_y, test_y):
        try:
            self.dtc = self.get_params_for_dtc(train_x, train_y)
            self.predict_dtc = self.dtc.predict(test_x)
            self.predict_prob_dtc = self.dtc.predict_proba(test_x)#[:,1]
#             print(self.predict_prob_dtc)
            self.conf_mat = confusion_matrix(test_y, self.predict_dtc)
            print(f"Confusion matrix: \n {self.conf_mat}")
            print(classification_report(test_y,self.predict_dtc))
            #if len(test_y.unique()) == 1: #if there is only one label in y, then roc_auc_score returns error. We will use accuracy in that case
            self.dtc_score = accuracy_score(test_y, self.predict_dtc)
            print(f'Accuracy for DecisionTreeClassifier:{str(self.dtc_score)}')
            #else:
            
            lb = LabelBinarizer()
            lb.fit(test_y)
            y_test = lb.transform(test_y)
            y_pred = lb.transform(self.predict_dtc)
            self.dtc_AUC_score = roc_auc_score(y_test, y_pred, average='macro')
#             auc={}
#             auc += roc_auc_score(test_y, self.predict_prob_dtc,  multi_class='ovo')
            print(f'AUC for DecisionTreeClassifier: {self.dtc_AUC_score}')
#             fpr , tpr , thresholds   = roc_curve(test_y, self.predict_dtc)
#             auc_score = auc(fpr,tpr) same as roc_auc_score for each class
            return self.dtc
        except Exception as e:
            print(f'Exception occured in dtc_model methods. Exception message: {str(e)}')
            raise Exception()            

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X.values,y, test_size=0.2, random_state=123)
dtc = DecisionTree_Classifier()
dtc.dtc_model(X_train, X_test, y_train, y_test)

Exception occured in get_params_for_dtc method. Exception message: could not convert string to float: '5more'
LogisticReg Parameter tuning  failed.
Exception occured in dtc_model methods. Exception message: 'NoneType' object has no attribute 'predict'


Exception: 

In [58]:
X.columns

Index(['doors', 'persons', 'buying_low', 'buying_med', 'buying_vhigh',
       'maintenance_low', 'maintenance_med', 'maintenance_vhigh',
       'lug_boot_med', 'lug_boot_small', 'safety_low', 'safety_med'],
      dtype='object')

In [59]:
X_train

array([['5more', '4', 0, ..., 0, 1, 0],
       ['2', '4', 0, ..., 1, 0, 1],
       ['2', 'more', 0, ..., 1, 0, 1],
       ...,
       ['2', '2', 1, ..., 1, 0, 0],
       ['5more', '4', 1, ..., 0, 1, 0],
       ['2', 'more', 1, ..., 0, 0, 1]], dtype=object)